# Bitcoin as "Digital Gold"? A Visual Analysis of Its Role as a Hedge Against Inflation and Market Volatility

As inflation creates general unease within our economy, we're seeing a rise in investments into alternative commodities and assets that are said to "hedge"—an investment that's expected to hold or increase its value over time, even as the cost of everything else goes up. One of these assets is Bitcoin ($BTC), a cryptocurrency often dubbed "Digital Gold" based on popular sentiment. 

As governments printed significant amounts of money, especially in recent years, many investors have looked for assets that can protect their wealth from the resulting inflation. Proponents claim Bitcoin, with its fixed supply, is a perfect candidate. A good hedge should ideally be stable, but what if Bitcoin becomes more chaotic and unpredictable precisely when you need it to be a safe haven? This project explores and investigates this popular claim by asking the question:

- **Does Bitcoin's volatility change during periods of high vs. low inflation?**
- **During major stock market crashes, does Bitcoin act as a 'safe haven' by holding its value, or does it crash even harder?**
- **How does Bitcoin’s performance change during spikes in market-wide fear and uncertainty, and does it behave more like gold (a hedge) or the S&P 500 (a risk asset)?**
- **Does Bitcoin behave like a hedge or a risk asset when U.S. interest rates (DFF) rise or fall, and how does its response compare to gold and the S&P 500?**

## Mining Metrics: Sourcing Our Datasets
Several key financial and economic datasets are needed to investigate the relationship between Bitcoin, inflation, and broader market behavior. The following data was sourced from Yahoo Finance (via the `yfinance` library) and the Federal Reserve Economic Data (FRED) database.

**1. Bitcoin (BTC-USD)**
> This dataset contains the daily price history of Bitcoin valued in U.S. Dollars, which is central to evaluating its performance as an asset.

- **Source:** Yahoo Finance (`yfinance` Ticker: BTC-USD)
- **Link:** https://finance.yahoo.com/quote/BTC-USD/
- **Features**:
    - **Date:** The trading day
    - **Open/High/Low/Close:** The opening, highest, lowest, and closing prices for the day.
    - **Volume:** The total number of Bitcoins traded.

**2. S&P 500 Index (^GSPC)**
> This data tracks the performance of 500 of the largest publicly-traded companies in the United States, offering a snapshot of the overall health of the U.S. stock market. This is essential for analyzing how Bitcoin behaves during broad market movements and crashes.

- **Source:** Yahoo Finance (`yfinance` Ticker: ^GSPC)
- **Link:** https://finance.yahoo.com/quote/GSPC/
- **Features:** Includes the same trading day, OHLC (Open, High, Low, Close) and Volume data points as the Bitcoin dataset, but for the S&P 500 index.

**3. CBOE Volatility Index (^VIX)**
> Also known as a "fear index," the VIX measures expected market volatility. It is crucial for understanding how Bitcoin's own volatility and price action correlate with periods of market fear and uncertainty.

 - **Source:** Yahoo Finance (`yfinance` Ticker: ^VIX)
 - **Link:** https://finance.yahoo.com/quote/^VIX/
 - **Features:** Contains OHLC data points representing the daily values of the index.

**4. Gold Futures (GC=F)**
> This dataset tracks the price of gold, the traditional safe-haven asset. It provides a direct benchmark to compare against Bitcoin's performance as an inflation hedge and store of value.

 - **Source:** Yahoo Finance (`yfinance` Ticker: GC=F)
 - **Link:** https://finance.yahoo.com/quote/GC=F/
 - **Features:** Contains OHLC and Volume data for gold futures contracts.

**5. U.S Inflation (CPIAUCSL)**
> This is the primary measure of inflation. The dataset tracks the average change in prices paid by urban consumers for a basket of goods and services.

 - **Source:** Federal Reserve Economic Data (FRED Ticker: CPIAUCSL)
 - **Link:** https://fred.stlouisfed.org/series/CPIAUCSL
 - **Features:**
    - **Date:** The date of the observation
    - **Value:** A seasonally adjusted index value (1982-1984 = 100) representing the relative cost of goods.

**6. Effective Federal Funds Rate (DFF)**
> This dataset shows the main interest rate set by the U.S. Federal Reserve. It’s the rate that strongly influences how expensive or cheap it is to borrow money in the economy. When this rate changes, it often affects things like mortgages, credit cards, and the stock market.

 - **Source:** Federal Reserve Economic Data (FRED Ticker: DFF)
 - **Link:** https://fred.stlouisfed.org/series/DFF
 - **Features:**
    - **Date:** The date of the observation
    - **Value:** The effective federal funds rate, expressed as a percentage.

## Panning for Gold: Preprocessing the Datasets

The goal is to create a clean dataset in which all variables are aligned in time and transformed into metrics for analysis.

This involves four main preprocessing steps:


**Step 1: Loading the Datasets**

We need to load all six datasets into our Jupyter Notebook. We'll use a common date range to ensure we're looking at the same period for all assets.

We'll import the necessary Python libraries (`pandas`, `yfinance`, `pandas_datareader`, `pandas-ta`, `numpy`, `datetime`, `matplotlib`, `seaborn`) and download the time series for each of the six datasets, storing them in pandas DataFrames.
- **pandas**: Core library for data handling and manipulation. We use it to load datasets, clean them, align dates, and calculate returns.  
- **yfinance**: Allows us to pull historical market data (e.g., Bitcoin, S&P 500, Gold) directly from Yahoo Finance.  
- **pandas_datareader**: Used to fetch economic data from sources like FRED (e.g., CPI, Effective Federal Funds Rate, VIX).  
- **pandas_ta**: A library of technical analysis indicators. While not heavily used here, it’s useful for calculating moving averages, volatility, or other trading metrics if needed.  
- **numpy**: Provides fast numerical operations, which support calculations like cumulative returns, percentage changes, and other array-based computations.  
- **datetime**: Used to handle and manipulate dates. This is essential for aligning datasets on a common timeline, resampling data into weekly or monthly periods, and clearly defining event windows.
- **matplotlib**: A versatile plotting library for building visualizations. We use it for detailed control over charts, adding labels, annotations, and customizing plots.  
- **seaborn**: A visualization library built on top of Matplotlib. We use it to create clean, high-level statistical plots (boxplots, bar charts, and line graphs).  

To keep our analysis consistent across all datasets, we first resample the raw CPI data to a clean monthly frequency (`MS`). From this standardized monthly series, we are then able to calculate the year-over-year (YoY) inflation rate. This transformation allows us to directly compare inflation trends with the financial assets (Bitcoin, S&P 500, and Gold) on the same timeline, ensuring that our results reflect meaningful economic shifts.

In [1]:
%pip install pandas yfinance pandas_datareader pandas-ta numpy datetime matplotlib seaborn

  Using cached yfinance-0.2.65-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached pandas_datareader-0.10.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached pandas_ta-0.4.67b0-py3-none-any.whl.metadata (2.2 kB)
  Using cached DateTime-5.5-py3-none-any.whl.metadata (33 kB)
  Using cached multitasking-0.0.12-py3-none-any.whl
  Using cached peewee-3.18.2-cp313-cp313-macosx_11_0_arm64.whl
  Using cached curl_cffi-0.13.0-cp39-abi3-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached websockets-15.0.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached numba-0.61.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.7 kB)
  Using cached pandas-2.3.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached numpy-2.2.6-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
Using cached yfinance-0.2.65-py2.py3-none-any.whl (119 kB)
Using cached pandas_datareader-0.10.0-py3-none-any.whl (109 kB)
Using cached pandas_ta-0.4.67b0-py3-none-any.whl (240 kB)
Using cached numba-0.61.2

**Step 2: Resampling, Merging and Handling Missing Values (NaNs)**

The market data (BTC, S&P 500, VIX, Gold) and the policy-related rate (DFF) are daily, while the core inflation data (CPI) is monthly. We need to get everything onto a single daily timeline.

We'll select a single, representative column from each of the daily market datasets (e.g., `Close`). Then, we'll merge the daily datasets into one primary DataFrame using the date as the common index.

 We merge this monthly rate into our daily DataFrame and use a method called forward-filling (`ffill`). This is the correct approach because the inflation rate for a given month is considered the prevailing rate for the entire month until the next value is announced. The `ffill` method achieves this by carrying the last valid observation forward to fill any subsequent gaps (e.g., a series like `[10, NaN, NaN, 15]` becomes `[10, 10, 10, 15]`).

**Step 3: Feature Engineering**

Raw prices and index values aren't always the best for analysis. So we need to transform them into more insightful metrics that directly addresses the research questions. We engineer and create the following new feautures:
- **Calculate Daily Returns**: For Bitcoin, the S&P 500, and Gold, calculate the daily percentage change. Daily returns normalize the data. A $100 price change means something very different for Bitcoin than it does for the S&P 500. Percentage returns allow us to compare their performance on an equal footing.
- **Calculate Normalized Bitcoin Volatility (ATR %)**: We calculate a Normalized Average True Range (ATR) for a robust measure of volatility. First, we find the 30-day ATR, which considers the daily high, low, and previous close to determine an asset's true trading range in dollars. This dollar value is then divided by the closing price to express volatility as a percentage, making it consistent and comparable.
- **Calculate Normalized Gold Volatility (ATR %)**: We calculate the same Normalized Average True Range (ATR) calculation to Gold. This provides a comparable percentage-based measure of Gold's true daily volatility, allowing for a direct comparison against Bitcoin in different market environments.

To calculate volatility for Gold and Bitcoin cleanly, we first pull out the High, Low, and Close columns for Bitcoin and Gold and rename them (for example, `High_btc`, `Low_btc`, `Close_btc`). These renamed columns are temporary and only exist so we can run the ATR calculation without confusing them with other assets. Once we’ve used them to calculate the volatility percentage, we drop these extra columns to keep the dataset tidy and focused only on the final features we need.

**Step 4: Finalizing the Dataset**

Finally, we'll clean up our DataFrame by selecting only the columns we need and dropping any initial rows that might have `NaN` values due to the rolling calculations.

Our final DataFrame will contain:
- Daily returns
- VIX index
- Inflation rate
- Fed Funds Rate
- Bitcoin's Rolling Volatility 

We'll then use `.dropna()` to remove any incomplete rows at the beginning.


## Charting the "Digital Gold" Thesis

Using the Seaborn, Matplotlib, and NumPy libraries, we will generate a series of targeted visualizations. Each plot is specifically designed to answer the research questions proposed in this project and reveal whether Bitcoin behaves more like a hedge or a risk asset in different economic conditions.  

### A Test of Stability: BTC Volatility vs. Inflation:

A core tenet of the "Digital Gold" thesis is that Bitcoin should act as a stable store of value, particularly when economic conditions are uncertain. This section puts that idea to the test by investigating a key question: Does Bitcoin's price stability/volatility change during periods of high inflation?

To provide a clear answer, we first analyze Bitcoin's performance in isolation. We grouped the data into three distinct inflation environments: Low (<2%), Moderate (2-5%), and High (>5%). A bar chart visualizes the average 30-day volatility—measured by the Normalized Average True Range (ATR)—for Bitcoin within each environment.

The results show a clear positive correlation. During periods of Low Inflation, Bitcoin's average daily price range was 3.99%. This volatility increased to 4.33% in Moderate Inflation environments and peaked at 5.00% during periods of High Inflation. This trend suggests that, contrary to the idea of it being a stable haven, Bitcoin's price becomes progressively less stable and more volatile as inflationary pressures increase.

To directly test the "Digital Gold" claim, we then expand this analysis with a comparative bar chart. This second visualization places Bitcoin's average volatility side-by-side with that of Gold—the benchmark traditional hedge against inflation—allowing for a direct comparison of their stability under different economic pressures, allowing for a direct comparison of their stability under different economic pressures.

The results reveal a stark contrast in how Bitcoin and Gold behave under different inflationary pressures. During periods of Low Inflation (<2%), Bitcoin's average daily volatility was 3.98%, already substantially higher than Gold's 1.18%.

As inflation increased, this divergence became more pronounced. In High Inflation (>5%) environments, Bitcoin's volatility rose to 5.03%, while Gold's volatility remained low and stable at 1.06%.

This trend demonstrates that while Gold maintains its characteristic stability as a traditional hedge, Bitcoin becomes progressively less stable as inflationary pressures increase. The data challenges the "Digital Gold" narrative by showing that Bitcoin's behavior is the opposite of the stability exhibited by its physical counterpart in the environments where a safe haven is needed most.

**So, Does Bitcoin's volatility change during periods of high vs. low inflation?**

Bitcoin's volatility does **NOT** remain stable but instead shows a distinct positive correlation with inflation. Rather than acting as a stabilizing force, the data reveals that Bitcoin becomes progressively more chaotic and unpredictable as inflationary pressures rise. This behavior is in stark contrast to that of Gold, which maintained low volatility across all environments. Ultimately, the findings suggest that during the very times an investor might seek a stable safe haven from inflation, Bitcoin's own price instability intensifies, challenging its effectiveness in that role.

### Bitcoin vs. Market Crashes

A primary argument for Bitcoin is its potential to serve as a 'safe haven'—an asset expected to retain or even increase its value during a stock market downturn. This analysis directly investigates this claim by visually comparing Bitcoin's performance against the S&P 500 during periods of significant market stress. We begin with a long-term view of their cumulative returns to establish context, before zooming in on two critical case studies: the acute, rapid panic of the March 2020 COVID-19 crash and the sustained pressure of the 2022 bear market.

From a broad, long-term perspective, the chart reveals two primary characteristics. The most striking is Bitcoin's massive outperformance over the S&P 500, showing a vastly greater growth in value over the entire period.

However, this outperformance was accompanied by significantly higher volatility. This is visually evident in the much sharper peaks and deeper drawdowns of the Bitcoin line compared to the relatively smoother growth of the stock market. A general inspection also suggests a tendency for the two assets to move in the same general direction, indicating that broad market upturns and downturns appear to affect both.

Let us zoom into the COVID-19 Crash from Feb-May 2020:

Bitcoin did not function as a traditional "safe haven" during the COVID-19 crash. Both Bitcoin and the S&P 500 experienced sharp drawdowns in March 2020, with Bitcoin initially falling even harder. However, Bitcoin’s recovery was much stronger, ultimately outperforming the S&P 500 by May. This indicates that while Bitcoin does not shield investors from immediate market turmoil, it can act as a high-risk, high-reward asset that rebounds more aggressively.

Let us zoom into the 2022 Bear Market:

Bitcoin did not behave like a "safe haven" during the 2022 bear market. While the S&P 500 declined steadily but moderately throughout the year, Bitcoin suffered a far steeper and more prolonged drop, losing more than half its value. Unlike the COVID-19 crash of 2020—where Bitcoin rebounded strongly after its initial fall—this time it failed to recover meaningfully and instead remained depressed. This performance suggests that Bitcoin amplifies downside risk during prolonged market stress rather than preserving value.

**So, During major stock market crashes, does Bitcoin act as a 'safe haven' by holding its value, or does it crash even harder?**

The evidence across both case studies suggests that Bitcoin does not act as a traditional "safe haven" during major stock market crashes. In the short-lived COVID-19 panic of March 2020, Bitcoin initially plunged even harder than equities but then staged a rapid and powerful rebound, ultimately outperforming the S&P 500. In contrast, during the prolonged 2022 bear market, Bitcoin suffered a far steeper and more sustained decline, losing over half its value while the S&P 500 experienced a more moderate drawdown.  

This mixed behavior suggests that Bitcoin is not a protective asset during market stress, but rather one that behaves with amplified sensitivity to market conditions. In other words, it tends to drop when overall risk sentiment turns negative, often by a larger amount than traditional stocks. Its eventual performance depends on the type of downturn: if the shock is short-lived, Bitcoin can bounce back quickly as speculative interest returns; but if the downturn drags on, it struggles to recover and remains suppressed.

Thus, while Bitcoin can deliver outsized gains in recovery phases, it does not consistently protect value during downturns. Its volatility and correlation with risk assets mean it amplifies exposure rather than insulating portfolios, challenging the narrative of Bitcoin as a reliable "safe haven."

### A Test of Resilience: BTC in Times of Fear

To test Bitcoin's ability to be a hedge, we turn to the VIX, often called the “fear index.” The VIX measures expected market volatility by tracking option prices (contracts that let investors bet on or protect against future price moves) on the S&P 500, making it one of the most widely used gauges of investor anxiety and uncertainty.  

In this section, we focus on periods when the VIX is elevated—moments of heightened fear—and compare the performance of Bitcoin, gold, and the S&P 500. To visualize this, we construct cumulative return indices (growth of \$1) and overlay shaded regions during high-VIX episodes. This approach makes it clear whether Bitcoin aligns more closely with traditional hedges (like gold) or with risk assets (like equities) in times of fear.  

The shaded regions highlight periods when the VIX (the “fear index”) was elevated, signaling heightened investor anxiety. During these spikes in market-wide fear, Bitcoin does not behave like a traditional safe haven. Instead, its performance more closely resembles that of stocks, with sharp drawdowns that often coincide with stress in the S&P 500.  

Gold, by contrast, remains far more stable during these same episodes, occasionally rising or at least holding its value, which is consistent with its reputation as a hedge. Bitcoin’s tendency to fall alongside equities—and at times by an even greater magnitude—suggests that it functions more as a high-risk, speculative asset rather than a defensive one.  

**So, how does Bitcoin’s performance change during spikes in market-wide fear and uncertainty, and does it behave more like gold (a hedge) or the S&P 500 (a risk asset)?**

Bitcoin’s performance during spikes in market-wide fear and uncertainty does not resemble that of a hedge. Instead, it behaves more like a risk asset, similar to the S&P 500. When the VIX signals elevated fear, Bitcoin tends to experience sharp declines that align closely with stress in equities. Gold, on the other hand, often maintains stability or even rises during these same periods, reinforcing its role as a hedge.

In short, when fear dominates markets, Bitcoin aligns with risk assets like the S&P 500 rather than hedges like gold. Its long-term potential for growth is significant, but in the short term it does not provide the protection investors typically seek during periods of elevated uncertainty.  

### Monetary Policy vs. BTC

Another important question for Bitcoin’s role as “Digital Gold” is how it reacts when U.S. interest rates change. Interest rates here are measured by the Effective Federal Funds Rate (DFF), which is the short-term rate banks charge each other for overnight loans. The Federal Reserve adjusts this rate as part of its monetary policy: raising it usually means borrowing money becomes more expensive (tightening financial conditions), while lowering it makes borrowing cheaper (loosening financial conditions).  

To answer this, we build a bar chart of average weekly returns under two regimes: when interest rates are above their median (high-rate) and when they are below (low-rate). For each regime, we calculate the average return of Bitcoin, gold, and the S&P 500. If Bitcoin behaves like gold, its returns should stay steady across regimes; if it behaves like a risk asset, its returns should drop during high-rate periods.  

This chart shows how Bitcoin, gold, and the S&P 500 perform when U.S. interest rates are high compared to when they are low. Interest rates here are measured by the Federal Funds Rate (DFF), which is the short-term rate set by the Federal Reserve to guide borrowing costs in the economy.  

What stands out most is that Bitcoin earns much higher average returns when interest rates are low, but its returns shrink when interest rates rise. In other words, Bitcoin seems to thrive when money is cheap and liquidity is plentiful, but it struggles once financial conditions tighten. Gold, on the other hand, remains more stable across both environments, showing the kind of steady behavior often expected from a hedge. The S&P 500, which represents the stock market, stays roughly the same in both high-rate and low-rate regimes, serving as a useful baseline for comparison.  

Taken together, this suggests that Bitcoin does not act like a hedge in the way gold does. Instead, it behaves more like a risk asset: strong gains when conditions are easy, but weaker performance when policy tightens. Rather than offering protection, Bitcoin’s fortunes seem tied to the broader cycle of risk and liquidity in the economy.  

**So, Does Bitcoin behave like a hedge or a risk asset when U.S. interest rates (DFF) rise or fall, and how does its response compare to gold and the S&P 500?**

Bitcoin behaves more like a risk asset rather than a hedge when U.S. interest rates rise or fall. Its performance is highly sensitive to monetary conditions: it thrives during periods of low rates when money is cheap and liquidity is abundant, but weakens when rates rise and financial conditions tighten. Gold, in contrast, remains stable across both high- and low-rate environments, reflecting its role as a hedge.

Taken together, Bitcoin’s dependency on liquidity makes it align more with equities than with a protective hedge like gold.

## Altogether:

These results tell a consistent story: Bitcoin behaves much more like a **risk asset** than a hedge. Its fortunes rise when liquidity is abundant and risk appetite is strong, but it struggles under inflation, monetary tightening, or market fear. Gold continues to serve as the safer hedge, while Bitcoin acts more like a speculative asset whose performance is tied to broader cycles of risk and liquidity.  

## External References  

1. **Inflation and Bitcoin**  
   - [Federal Reserve CPI data (CPIAUCSL)](https://fred.stlouisfed.org/series/CPIAUCSL)  
   - [Academic study on Bitcoin and inflation hedging](https://www.sciencedirect.com/science/article/pii/S1544612321003810)  

2. **Bitcoin in Market Crashes**  
   - [S&P 500 historical data](https://finance.yahoo.com/quote/GSPC/)  
   - [Analysis of Bitcoin’s performance in the COVID-19 crash](https://www.sciencedirect.com/science/article/pii/S1059056025006756)  

3. **Bitcoin vs. Market Fear (VIX)**  
   - [CBOE Volatility Index (VIX)](https://finance.yahoo.com/quote/^VIX/)  
   - [Bitcoin and investor sentiment research](https://www.sciencedirect.com/science/article/pii/S1544612319309274)  

4. **Interest Rates (DFF) and Bitcoin**  
   - [Effective Federal Funds Rate (DFF)](https://fred.stlouisfed.org/series/DFF)  
   - [Study on monetary policy and cryptocurrency](https://www.sciencedirect.com/science/article/abs/pii/S0261560623000815)  

5. **Gold as a Hedge (Comparison Benchmark)**  
   - [Gold Futures data](https://finance.yahoo.com/quote/GC=F/)  
   - [Classic paper on gold as a hedge and safe haven](https://www.sciencedirect.com/science/article/abs/pii/S154461231100002X)  
